# Projet EMA

Ce projet est réalisé dans le cadre du cours Sécurité des composants à l'ENSTA Bretagne.

Source des données : (sur mon Google Drive pour l'instant) => penser à mettre le lien.

In [24]:
import matplotlib.pyplot as plt
import os
import re
import numpy as np
# import modin.pandas as pd
import pandas as pd

os.environ['MODIN_ENGINE'] = 'ray'  # Modin will use Ray

In [27]:
directory = os.path.join(os.getcwd(), 'TRACES_DATA')
tracesFiles = os.listdir(directory)
df = pd.read_csv(os.path.join(directory, tracesFiles[0]), sep=',', header=None)
df.head(5)
# show the first 5 rows

,0,1,2,3,4,5,6,7,8,9,...,3990,3991,3992,3993,3994,3995,3996,3997,3998,3999
0,0.003253,0.003226,0.003206,0.003196,0.003189,0.003189,0.003193,0.003203,0.003213,0.003223,...,0.002434,0.00249,0.002554,0.002627,0.002703,0.00279,0.002883,0.002986,0.003093,0.003209


## Récupération des informations dans le nom des fichiers

Les fichiers sont de la forme : `trace_AES_0000001_key=4c8cdf23b5c906f79057ec7184193a67_pti=8b5061cad6ae6aa4369eac99585d2bcb_cto=554b407d3634156c081abe31752ebd02.csv`


In [7]:
keys = []
pti = []
cto = []
regex_filename = re.compile(
    r'\w*key=([a-z0-9]+)_pti=([a-z0-9]+)_cto=([a-z0-9]+).csv')
for filename in tracesFiles:
    # print(filename)
    matchfilename = regex_filename.match(filename)
    if (matchfilename):
        keys.append(matchfilename.group(1))
        pti.append(matchfilename.group(2))
        cto.append(matchfilename.group(3))
    # print(matchCto)
print(len(keys), len(pti), len(cto))


20000 20000 20000


On veut des matrices de la forme :

- keys : 20 000 lignes, 16 colonnes
- pti : 20 000 lignes, 16 colonnes
- cto : 20 000 lignes, 16 colonnes
- traces : 20 000 lignes, 4 000 colonnes


Ensuite, on les enregistre dans leur fichier respectif.

In [29]:
# Convertir une donnée en binaire
data = keys[1]
print(data)
print(int(data, 16))
res = [int(data[i:i+2], 16) for i in range(0, len(data), 2)]
print(res)
int(data[:2], 16)



4c8cdf23b5c906f79057ec7184193a67
101752775046435598298791685242631043687
[76, 140, 223, 35, 181, 201, 6, 247, 144, 87, 236, 113, 132, 25, 58, 103]


76

In [16]:
traces = np.zeros((0, 4000))

for filename in tracesFiles[:1000]:
    fullPath = os.path.join(directory, filename)
    values = np.loadtxt(fullPath, dtype=float, delimiter=',')
    traces = np.append(traces, values.reshape((1, 4000)), axis=0)
    # print("Traces : ", values[:10], "avec", len(values), "valeurs")

print(traces.shape)
print(traces[:10])


(1000, 4000)
[[ 3.2526e-03  3.2260e-03  3.2060e-03 ...  2.9863e-03  3.0928e-03
   3.2093e-03]
 [ 2.1939e-03  2.1973e-03  2.2039e-03 ...  2.7399e-03  2.7066e-03
   2.6634e-03]
 [ 2.2672e-03  2.2971e-03  2.3238e-03 ...  3.7953e-04  2.5302e-04
   1.2984e-04]
 ...
 [ 1.7478e-03  1.7645e-03  1.7844e-03 ... -1.3983e-04 -6.6584e-05
   3.3292e-06]
 [-7.2909e-04 -7.1245e-04 -6.8914e-04 ... -1.2651e-03 -1.2451e-03
  -1.2218e-03]
 [-2.2905e-03 -2.1740e-03 -2.0475e-03 ... -1.0920e-03 -1.0387e-03
  -9.8211e-04]]


In [4]:
df = pd.DataFrame()
for filename in tracesFiles[:100]:
    df = pd.read_csv(os.path.join(directory, filename), sep=',')
df



KeyboardInterrupt



In [36]:
# Initialisation de la matrice pour stocker les données
data_matrix = np.empty((0,4))
data_pandas = pd.DataFrame()

def readcsv_numpy(filename, data_matrix):
    data = np.genfromtxt(filename, delimiter=",")
    data_matrix = np.append(data_matrix.reshape(data_matrix.shape[0],data.shape[0]), data.reshape(1,data.shape[0]), axis=0)
    return data_matrix
def readcsv_pandas(filename:str, data_pandas: pd.DataFrame):
    data = pd.read_csv(filename, sep=',', header=None)
    data_pandas = pd.concat([data_pandas, data], axis=0)
    return data_pandas

# Boucle pour lire les fichiers CSV
for file in tracesFiles[:100]:
    # Test with numpy
    # data_matrix = readcsv_numpy(os.path.join(directory, file), data_matrix)
    # Test with pandas
    data_pandas = readcsv_pandas(os.path.join(directory, file), data_pandas)

# Afficher la matrice
print(data_matrix[:10])

KeyboardInterrupt: 